# Makes predictions of unlabeled images

In [13]:
# import libraries and dependencies
import tensorflow as tf 
import tensorflow_hub as hub
import os
import numpy as np 
import matplotlib.pyplot as plt
import shutil

In [8]:
test_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/sample_images/'
img_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/sample_images/images/'
pred_dir = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/predictions/'
multi_model_path = 'F:/Nilgai_photo_database/Nilgai Classifier/Nilgai/models/Final_multi_05262020.h5'

BATCH_SIZE = 1
IMG_HEIGHT = 299
IMG_WIDTH = 299

def img_crop(img):
    # for cropping
    offset_width = 0
    offset_height = 9
    target_height = 273
    target_width = 299

    # for random width shift
    wrg = 0.5 
    hrg = 0.0
    row_axis =  2
    channel_axis = 1

    new_img = tf.image.crop_to_bounding_box(img, offset_height, offset_width, target_height, target_width)
    new_img = tf.image.resize(new_img, (299,299))
    return new_img

img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, preprocessing_function=img_crop)

test_generator = img_generator.flow_from_directory(directory=str(test_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode= None)
# load model
multi_model = tf.keras.models.load_model(multi_model_path, custom_objects={'KerasLayer':hub.KerasLayer}, compile=True)

Found 100 images belonging to 1 classes.


In [9]:
# make predictions
pred_steps=np.ceil(test_generator.samples/test_generator.batch_size)

pred = multi_model.predict(test_generator, verbose=1, workers=4, steps=pred_steps)

100/100 [==============================] - 2s 18ms/step


NameError: name 'multi_model' is not defined

In [10]:
# define class names and create and index for future mapping
class_names = [('Armadillo', 0),
 ('Birds', 1),
 ('Bobcat', 2),
 ('Cattle', 3),
 ('Coyote', 4),
 ('Dog', 5),
 ('Exotics, Other', 6),
 ('Horse', 7),
 ('Humans', 8),
 ('Mouse', 9),
 ('Nilgai', 10),
 ('None', 11),
 ('Ocelot', 12),
 ('Opossum', 13),
 ('Pig', 14),
 ('Rabbit', 15),
 ('Raccoon', 16),
 ('Rat', 17),
 ('Skunk', 18),
 ('Spider', 19),
 ('Squirrel', 20),
 ('Tortoise', 21),
 ('Turkey', 22),
 ('Unknown', 23),
 ('White-tailed Deer', 24)]
class_names = np.array([key.title() for key, value in class_names])
predicted_id = np.argmax(pred, axis=-1)
predicted_label_batch = class_names[predicted_id]

In [11]:
for i in class_names:
    img_path = os.path.join(pred_dir, str(i))
    try:
        os.makedirs(img_path, exist_ok=False)
    except:
        print("The " + '"' + str(i) + '"' + " folder already exists. Check your prediction path.")

The "Nilgai" folder already exists. Check your prediction path.


In [14]:
## Map prediction name to labels, move images to repsective folders, and print results
## Todo: make this a function
for i, logits in enumerate(pred):
    class_idx = tf.argmax(logits).numpy()
    p = logits[class_idx]
    name = class_names[class_idx]
    img_name = os.path.split(test_generator.filenames[i])[1]
    samp_name = os.path.splitext(img_name)[0]
    img = os.path.join(img_dir,img_name)
    dst = os.path.join(pred_dir, name)
    shutil.copy(img, dst)
    print("\n{} prediction: {} ({:4.2f}%)".format(samp_name, name, 100*p))
    print("Moved to Nilgai/predictions/" + name + ".")



0 prediction: Armadillo (100.00%)
Moved to Nilgai/predictions/Armadillo.

1 prediction: Cattle (100.00%)
Moved to Nilgai/predictions/Cattle.

10 prediction: Tortoise (100.00%)
Moved to Nilgai/predictions/Tortoise.

11 prediction: Rabbit (100.00%)
Moved to Nilgai/predictions/Rabbit.

12 prediction: Ocelot (100.00%)
Moved to Nilgai/predictions/Ocelot.

13 prediction: Armadillo (100.00%)
Moved to Nilgai/predictions/Armadillo.

14 prediction: None (100.00%)
Moved to Nilgai/predictions/None.

15 prediction: Raccoon (100.00%)
Moved to Nilgai/predictions/Raccoon.

16 prediction: Skunk (100.00%)
Moved to Nilgai/predictions/Skunk.

17 prediction: Mouse (100.00%)
Moved to Nilgai/predictions/Mouse.

18 prediction: Mouse (100.00%)
Moved to Nilgai/predictions/Mouse.

19 prediction: Nilgai (96.29%)
Moved to Nilgai/predictions/Nilgai.

2 prediction: Exotics, Other (100.00%)
Moved to Nilgai/predictions/Exotics, Other.

20 prediction: Cattle (97.75%)
Moved to Nilgai/predictions/Cattle.

21 prediction: